In [1]:
import sys
import re
import pickle
import numpy as np
import random
import os

In [2]:
_PAD = "_PAD"
_GO = "_GO"
_EOS = "_EOS"
_UNK = "_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]

PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3

_WORD_SPLIT = re.compile("([.,!?\"':;)(])")
_DIGIT_RE = re.compile(R"\d")

In [3]:
def basic_tokenizer(sentence):
    """ Split sentence into list of tokens """
    words = []
    for space_separated_item in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_item))
    return [w for w in words if w] # if w removes the ""

def get_vocab(tokenized, max_vocab_size):
    """
    Get vocab_list, vocab_dict and rev_vocab_dict given the
    tokenized sentences.
    """
    # Replace word count
    vocab = {}
    for sentence in tokenized:
        for word in sentence:
            if word in vocab:
                vocab[word] += 1
            else:
                vocab[word] = 1
    vocab_list = _START_VOCAB + sorted(vocab, key=vocab.get, reverse=True)
    print("original vocab_len: "+str(len(vocab_list)))
    if len(vocab_list) > max_vocab_size:
        vocab_list = vocab_list[:max_vocab_size]

    # Get vocab dict (word -> token) and rev dict (token -> word)
    vocab_dict = dict([(x,y) for (y,x) in enumerate(vocab_list)])
    rev_vocab_dict = {v: k for k, v in vocab_dict.items()}

    return vocab_list, vocab_dict, rev_vocab_dict

def sentence_to_token_ids(sentence, vocab_dict, target_lang,
    normalize_digits=True):
    """
    Convert a single sentence of words to token ids. If it is the target
    language, we will append an EOS token to the end.
    """
    if not normalize_digits:
        # replace words not in vocab_dict with UNK_ID
        tokens = [vocab_dict.get(w, UNK_ID) for w in sentence]
    else:
        tokens = [vocab_dict.get(_DIGIT_RE.sub(b"0", w), UNK_ID)
            for w in sentence]

    # Append EOS token if target langauge sentence

    return tokens


def data_to_token_ids(tokenized, vocab_dict, normalize_digits=True):
    """
    Convert tokens into ids used vocab_dict and normalize all digits
    to 0.
    """
    data_as_tokens = []
    #max_len = max(len(sentence) for sentence in tokenized) + 1 # +1 for EOS
    #max_len=max_seq_len+1
    for sentence in tokenized:
        token_ids = sentence_to_token_ids(sentence, vocab_dict, normalize_digits)
        data_as_tokens.extend(token_ids)
    return data_as_tokens

def process_data(datafile, max_vocab_size):
    """
    Read the sentences from our datafiles.
    """
    with open(datafile, 'rb') as f:
        sentences = pickle.load(f)

    # Split into tokens
    tokenized = []
    for i in range(len(sentences)):
        tokenized.append(basic_tokenizer(sentences[i]))

    # Get vocab information
    vocab_list, vocab_dict, rev_vocab_dict = get_vocab(tokenized,
        max_vocab_size)

    # Convert data to token ids
    data_as_tokens= data_to_token_ids(tokenized, vocab_dict, normalize_digits=True)
    
    data_mixed=[]
    for word in data_as_tokens:
        if word==vocab_dict["the"] or word==vocab_dict["a"]:
            data_mixed.append(vocab_dict["the"] if random.random()<0.5 else vocab_dict["a"])
        else:
            data_mixed.append(word)
            
    pickle.dump((data_as_tokens, vocab_dict,  rev_vocab_dict), open('preprocess.p', 'wb'))
    
    return data_as_tokens, data_mixed,vocab_dict, rev_vocab_dict

In [4]:
def compare_a_the(x_text,y_text):
    theid=vocab_to_int["the"]
    aid=vocab_to_int["a"]
    total=0
    mix=0
    for i,word in enumerate(x_text):
        if word==theid or word==aid:
            total+=1
            if x_text[i]!=y_text[i]:
                mix+=1
    return mix,total
        

In [5]:
int_label,int_text, vocab_to_int, int_to_vocab = \
        process_data('original.p', max_vocab_size=12000)

original vocab_len: 12251


In [6]:
mix,total=compare_a_the(int_label,int_text)
mix/total

0.5035501048894626

In [7]:
len(int_label)

200780

In [8]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs=tf.placeholder(tf.int32,[None,None],name='input')
    targets=tf.placeholder(tf.int32,[None,None],name='targets')
    learning_rate=tf.placeholder(tf.float32,name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    return inputs, targets, learning_rate,keep_prob

In [31]:
def get_init_cell(batch_size, rnn_size, keep_prob):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    def single_cell(keepprob):
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keepprob)
        return drop
                                         
    cell_fw = tf.contrib.rnn.MultiRNNCell([single_cell(keep_prob) for _ in range(1)])
    cell_bw = tf.contrib.rnn.MultiRNNCell([single_cell(keep_prob) for _ in range(1)])
    return cell_fw,cell_bw

In [32]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embeddings = tf.Variable(tf.random_normal([vocab_size, embed_dim], stddev=0.1), name='embeddings')
    embed = tf.nn.embedding_lookup(embeddings, input_data, name='embed')
    return embed

In [33]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    (cell_fw,cell_bw)=cell
    outputs, final_state = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell_fw,cell_bw=cell_bw, inputs=inputs, dtype=tf.float32)
    outputs = tf.concat(outputs, 2)
    final_state=tf.concat(final_state, 2)
    final_state = tf.identity(final_state, name='final_state')
    return (outputs, final_state)

In [34]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn=None)
    return (logits, final_state)

In [35]:
def get_batches(int_text,label_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    num_batches = len(int_text) // (batch_size * seq_length)
    xtext = int_text[:num_batches * batch_size * seq_length]
    ytext = label_text[:num_batches * batch_size * seq_length]
    
        
    xreshape=np.reshape(xtext,[batch_size,-1])
    yreshape=np.reshape(ytext,[batch_size,-1])
    batches = []
    
    for i in range(0,xreshape.shape[1],seq_length):
        xx=xreshape[:,i:i+seq_length]
        yy=yreshape[:,i:i+seq_length]
        batches.append([xx,yy])
        
    return np.asarray(batches)

In [36]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 200
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 30
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

pickle.dump((seq_length, save_dir), open('params.p', 'wb'))

In [37]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr,keep_prob = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell= get_init_cell(input_data_shape[0], rnn_size,keep_prob)
    logits,final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)
    
    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [38]:
split_index=int(0.8*len(int_text))
train_source = int_text[:split_index]
valid_source = int_text[split_index:]

train_label= int_label[:split_index]
valid_label= int_label[split_index:]

print(len(valid_label))
train_batches = get_batches(train_source,train_label, batch_size, seq_length)
valid_batches=get_batches(valid_source,valid_label, 800, seq_length)
valid_x=valid_batches[0][0]
valid_y=valid_batches[0][1]
#print(valid_x)
#print(valid_y)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):

        for batch_i, (x, y) in enumerate(train_batches):
            feed = {
                input_text: x,
                targets: y,
                lr: learning_rate,
                keep_prob:0.5}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(train_batches) + batch_i) % show_every_n_batches == 0:
                feed_valid={
                input_text: valid_x,
                keep_prob:1,
                targets: valid_y}
                valid_loss,= sess.run([cost], feed_valid)
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}  valid_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(train_batches),
                    train_loss,
                    valid_loss))
                
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

40156
Epoch   0 Batch    0/41   train_loss = 9.393  valid_loss = 9.383
Epoch   0 Batch   10/41   train_loss = 8.311  valid_loss = 7.977
Epoch   0 Batch   20/41   train_loss = 6.534  valid_loss = 6.605
Epoch   0 Batch   30/41   train_loss = 6.442  valid_loss = 6.587
Epoch   0 Batch   40/41   train_loss = 6.387  valid_loss = 6.438
Epoch   1 Batch    9/41   train_loss = 6.171  valid_loss = 6.279
Epoch   1 Batch   19/41   train_loss = 6.069  valid_loss = 6.133
Epoch   1 Batch   29/41   train_loss = 5.838  valid_loss = 5.959
Epoch   1 Batch   39/41   train_loss = 5.625  valid_loss = 5.764
Epoch   2 Batch    8/41   train_loss = 5.455  valid_loss = 5.567
Epoch   2 Batch   18/41   train_loss = 5.278  valid_loss = 5.398
Epoch   2 Batch   28/41   train_loss = 5.127  valid_loss = 5.250
Epoch   2 Batch   38/41   train_loss = 4.915  valid_loss = 5.099
Epoch   3 Batch    7/41   train_loss = 4.853  valid_loss = 4.946
Epoch   3 Batch   17/41   train_loss = 4.764  valid_loss = 4.795
Epoch   3 Batch   2

Epoch  30 Batch   30/41   train_loss = 0.213  valid_loss = 1.069
Epoch  30 Batch   40/41   train_loss = 0.185  valid_loss = 1.066
Epoch  31 Batch    9/41   train_loss = 0.179  valid_loss = 1.064
Epoch  31 Batch   19/41   train_loss = 0.189  valid_loss = 1.058
Epoch  31 Batch   29/41   train_loss = 0.174  valid_loss = 1.055
Epoch  31 Batch   39/41   train_loss = 0.174  valid_loss = 1.053
Epoch  32 Batch    8/41   train_loss = 0.145  valid_loss = 1.051
Epoch  32 Batch   18/41   train_loss = 0.169  valid_loss = 1.045
Epoch  32 Batch   28/41   train_loss = 0.176  valid_loss = 1.044
Epoch  32 Batch   38/41   train_loss = 0.153  valid_loss = 1.041
Epoch  33 Batch    7/41   train_loss = 0.153  valid_loss = 1.038
Epoch  33 Batch   17/41   train_loss = 0.138  valid_loss = 1.033
Epoch  33 Batch   27/41   train_loss = 0.157  valid_loss = 1.030
Epoch  33 Batch   37/41   train_loss = 0.127  valid_loss = 1.031
Epoch  34 Batch    6/41   train_loss = 0.128  valid_loss = 1.028
Epoch  34 Batch   16/41  

Epoch  61 Batch   29/41   train_loss = 0.009  valid_loss = 0.921
Epoch  61 Batch   39/41   train_loss = 0.011  valid_loss = 0.921
Epoch  62 Batch    8/41   train_loss = 0.010  valid_loss = 0.921
Epoch  62 Batch   18/41   train_loss = 0.011  valid_loss = 0.921
Epoch  62 Batch   28/41   train_loss = 0.011  valid_loss = 0.921
Epoch  62 Batch   38/41   train_loss = 0.012  valid_loss = 0.921
Epoch  63 Batch    7/41   train_loss = 0.013  valid_loss = 0.922
Epoch  63 Batch   17/41   train_loss = 0.010  valid_loss = 0.922
Epoch  63 Batch   27/41   train_loss = 0.012  valid_loss = 0.921
Epoch  63 Batch   37/41   train_loss = 0.010  valid_loss = 0.921
Epoch  64 Batch    6/41   train_loss = 0.011  valid_loss = 0.921
Epoch  64 Batch   16/41   train_loss = 0.010  valid_loss = 0.921
Epoch  64 Batch   26/41   train_loss = 0.010  valid_loss = 0.921
Epoch  64 Batch   36/41   train_loss = 0.009  valid_loss = 0.921
Epoch  65 Batch    5/41   train_loss = 0.008  valid_loss = 0.923
Epoch  65 Batch   15/41  

Epoch  92 Batch   28/41   train_loss = 0.004  valid_loss = 0.912
Epoch  92 Batch   38/41   train_loss = 0.003  valid_loss = 0.916
Epoch  93 Batch    7/41   train_loss = 0.004  valid_loss = 0.917
Epoch  93 Batch   17/41   train_loss = 0.005  valid_loss = 0.914
Epoch  93 Batch   27/41   train_loss = 0.006  valid_loss = 0.913
Epoch  93 Batch   37/41   train_loss = 0.004  valid_loss = 0.914
Epoch  94 Batch    6/41   train_loss = 0.003  valid_loss = 0.917
Epoch  94 Batch   16/41   train_loss = 0.004  valid_loss = 0.917
Epoch  94 Batch   26/41   train_loss = 0.004  valid_loss = 0.919
Epoch  94 Batch   36/41   train_loss = 0.004  valid_loss = 0.921
Epoch  95 Batch    5/41   train_loss = 0.003  valid_loss = 0.919
Epoch  95 Batch   15/41   train_loss = 0.004  valid_loss = 0.913
Epoch  95 Batch   25/41   train_loss = 0.004  valid_loss = 0.915
Epoch  95 Batch   35/41   train_loss = 0.004  valid_loss = 0.925
Epoch  96 Batch    4/41   train_loss = 0.005  valid_loss = 0.920
Epoch  96 Batch   14/41  

Epoch 123 Batch   27/41   train_loss = 0.004  valid_loss = 0.898
Epoch 123 Batch   37/41   train_loss = 0.002  valid_loss = 0.898
Epoch 124 Batch    6/41   train_loss = 0.002  valid_loss = 0.898
Epoch 124 Batch   16/41   train_loss = 0.002  valid_loss = 0.899
Epoch 124 Batch   26/41   train_loss = 0.002  valid_loss = 0.900
Epoch 124 Batch   36/41   train_loss = 0.002  valid_loss = 0.899
Epoch 125 Batch    5/41   train_loss = 0.002  valid_loss = 0.899
Epoch 125 Batch   15/41   train_loss = 0.002  valid_loss = 0.900
Epoch 125 Batch   25/41   train_loss = 0.001  valid_loss = 0.901
Epoch 125 Batch   35/41   train_loss = 0.002  valid_loss = 0.900
Epoch 126 Batch    4/41   train_loss = 0.003  valid_loss = 0.900
Epoch 126 Batch   14/41   train_loss = 0.002  valid_loss = 0.901
Epoch 126 Batch   24/41   train_loss = 0.001  valid_loss = 0.901
Epoch 126 Batch   34/41   train_loss = 0.001  valid_loss = 0.901
Epoch 127 Batch    3/41   train_loss = 0.002  valid_loss = 0.900
Epoch 127 Batch   13/41  

Epoch 154 Batch   26/41   train_loss = 0.001  valid_loss = 0.905
Epoch 154 Batch   36/41   train_loss = 0.001  valid_loss = 0.906
Epoch 155 Batch    5/41   train_loss = 0.001  valid_loss = 0.906
Epoch 155 Batch   15/41   train_loss = 0.001  valid_loss = 0.907
Epoch 155 Batch   25/41   train_loss = 0.001  valid_loss = 0.907
Epoch 155 Batch   35/41   train_loss = 0.001  valid_loss = 0.905
Epoch 156 Batch    4/41   train_loss = 0.001  valid_loss = 0.905
Epoch 156 Batch   14/41   train_loss = 0.001  valid_loss = 0.907
Epoch 156 Batch   24/41   train_loss = 0.001  valid_loss = 0.907
Epoch 156 Batch   34/41   train_loss = 0.001  valid_loss = 0.906
Epoch 157 Batch    3/41   train_loss = 0.001  valid_loss = 0.907
Epoch 157 Batch   13/41   train_loss = 0.001  valid_loss = 0.908
Epoch 157 Batch   23/41   train_loss = 0.001  valid_loss = 0.908
Epoch 157 Batch   33/41   train_loss = 0.001  valid_loss = 0.907
Epoch 158 Batch    2/41   train_loss = 0.000  valid_loss = 0.906
Epoch 158 Batch   12/41  

Epoch 185 Batch   25/41   train_loss = 0.000  valid_loss = 0.910
Epoch 185 Batch   35/41   train_loss = 0.001  valid_loss = 0.910
Epoch 186 Batch    4/41   train_loss = 0.000  valid_loss = 0.909
Epoch 186 Batch   14/41   train_loss = 0.000  valid_loss = 0.909
Epoch 186 Batch   24/41   train_loss = 0.000  valid_loss = 0.910
Epoch 186 Batch   34/41   train_loss = 0.000  valid_loss = 0.909
Epoch 187 Batch    3/41   train_loss = 0.000  valid_loss = 0.910
Epoch 187 Batch   13/41   train_loss = 0.000  valid_loss = 0.910
Epoch 187 Batch   23/41   train_loss = 0.000  valid_loss = 0.910
Epoch 187 Batch   33/41   train_loss = 0.000  valid_loss = 0.911
Epoch 188 Batch    2/41   train_loss = 0.000  valid_loss = 0.911
Epoch 188 Batch   12/41   train_loss = 0.000  valid_loss = 0.911
Epoch 188 Batch   22/41   train_loss = 0.000  valid_loss = 0.911
Epoch 188 Batch   32/41   train_loss = 0.000  valid_loss = 0.910
Epoch 189 Batch    1/41   train_loss = 0.000  valid_loss = 0.910
Epoch 189 Batch   11/41  

In [39]:
import tensorflow as tf
import numpy as np
import pickle

int_text, vocab_to_int, int_to_vocab= pickle.load(open('preprocess.p', mode='rb'))
seq_length, load_dir = pickle.load(open('params.p', mode='rb'))
token_original,len_original,token_obfused,len_obfused=pickle.load(open('twotaledumped.p', mode='rb'))

In [40]:
_PAD = "_PAD"
_GO = "_GO"
_EOS = "_EOS"
_UNK = "_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]

PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3

In [41]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return loaded_graph.get_tensor_by_name("input:0"), \
           loaded_graph.get_tensor_by_name("probs:0"),\
           loaded_graph.get_tensor_by_name("keep_prob:0"),

In [42]:
def compare_a_the(x_text,y_text,vocab_to_int):
    theid=vocab_to_int["the"]
    aid=vocab_to_int["a"]
    total=0
    mix=0
    for i,word in enumerate(x_text):
        if word==theid or word==aid:
            total+=1
            if x_text[i]!=y_text[i]:
                mix+=1
    return mix,total

In [43]:
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, probs,keep_prob = get_tensors(loaded_graph)
        
    mixnum=0
    totalnum=0
    for i,sentence_token in enumerate(token_obfused):
        if i%10==0:
            print("Proceeding {}/{}".format(i+1,len(token_obfused)))
        
        input_sentence = [vocab_to_int.get(word, vocab_to_int[_UNK]) for word in sentence_token]
        
        batch_shell = np.zeros((1, len(input_sentence)))
        batch_shell[0] = input_sentence
        chatbot_logits = sess.run(probs, {input_text: batch_shell,keep_prob:1})[0]
        
        original_sentence=  [vocab_to_int.get(word, vocab_to_int[_UNK]) for word in token_original[i]]
        predicted_sentence= [i for i in np.argmax(chatbot_logits, 1)]
        
        mixathe,totalathe=compare_a_the(original_sentence,predicted_sentence,vocab_to_int)
        totalnum+=totalathe
        mixnum+=mixathe
    print(1-mixnum/totalnum)
    

INFO:tensorflow:Restoring parameters from ./save
Proceeding 1/5809
Proceeding 11/5809
Proceeding 21/5809
Proceeding 31/5809
Proceeding 41/5809
Proceeding 51/5809
Proceeding 61/5809
Proceeding 71/5809
Proceeding 81/5809
Proceeding 91/5809
Proceeding 101/5809
Proceeding 111/5809
Proceeding 121/5809
Proceeding 131/5809
Proceeding 141/5809
Proceeding 151/5809
Proceeding 161/5809
Proceeding 171/5809
Proceeding 181/5809
Proceeding 191/5809
Proceeding 201/5809
Proceeding 211/5809
Proceeding 221/5809
Proceeding 231/5809
Proceeding 241/5809
Proceeding 251/5809
Proceeding 261/5809
Proceeding 271/5809
Proceeding 281/5809
Proceeding 291/5809
Proceeding 301/5809
Proceeding 311/5809
Proceeding 321/5809
Proceeding 331/5809
Proceeding 341/5809
Proceeding 351/5809
Proceeding 361/5809
Proceeding 371/5809
Proceeding 381/5809
Proceeding 391/5809
Proceeding 401/5809
Proceeding 411/5809
Proceeding 421/5809
Proceeding 431/5809
Proceeding 441/5809
Proceeding 451/5809
Proceeding 461/5809
Proceeding 471/5809
Pr

Proceeding 3941/5809
Proceeding 3951/5809
Proceeding 3961/5809
Proceeding 3971/5809
Proceeding 3981/5809
Proceeding 3991/5809
Proceeding 4001/5809
Proceeding 4011/5809
Proceeding 4021/5809
Proceeding 4031/5809
Proceeding 4041/5809
Proceeding 4051/5809
Proceeding 4061/5809
Proceeding 4071/5809
Proceeding 4081/5809
Proceeding 4091/5809
Proceeding 4101/5809
Proceeding 4111/5809
Proceeding 4121/5809
Proceeding 4131/5809
Proceeding 4141/5809
Proceeding 4151/5809
Proceeding 4161/5809
Proceeding 4171/5809
Proceeding 4181/5809
Proceeding 4191/5809
Proceeding 4201/5809
Proceeding 4211/5809
Proceeding 4221/5809
Proceeding 4231/5809
Proceeding 4241/5809
Proceeding 4251/5809
Proceeding 4261/5809
Proceeding 4271/5809
Proceeding 4281/5809
Proceeding 4291/5809
Proceeding 4301/5809
Proceeding 4311/5809
Proceeding 4321/5809
Proceeding 4331/5809
Proceeding 4341/5809
Proceeding 4351/5809
Proceeding 4361/5809
Proceeding 4371/5809
Proceeding 4381/5809
Proceeding 4391/5809
Proceeding 4401/5809
Proceeding 44

In [ ]:

input_sentence = ["I","saw","the","man",",","who","is","a","husband","of","me"]
    print(seq_length)
    input_sentence = [vocab_to_int.get(word, vocab_to_int[_UNK]) for word in input_sentence]  
    batch_shell = np.zeros((7, len(input_sentence)))
    batch_shell[0] = input_sentence
    chatbot_logits = sess.run(probs, {input_text: batch_shell})[0]
    print(chatbot_logits.shape)
    print('Input')
    print('  Word Ids:      {}'.format([i for i in input_sentence]))
    print('  Input Words: {}'.format([int_to_vocab[i] for i in input_sentence]))
    print('\nPrediction')
    print('  Word Ids:      {}'.format([i for i in np.argmax(chatbot_logits, 1)]))
    print('  Chatbot Answer Words: {}'.format([int_to_vocab[i] for i in np.argmax(chatbot_logits, 1)]))